# Dataset

In [5]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('train_dataset_train.csv', sep=';')

In [7]:
df.head()

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу


In [8]:
df.describe()

,Исполнитель,Группа тем,Текст инцидента,Тема
count,23128,23128,23128,23128
unique,10,26,22656,195
top,Министерство здравоохранения,Здравоохранение/Медицина,'Сообщение без текста,Дети и многодетные семьи
freq,5765,4673,134,2008


In [9]:
df['Исполнитель'].unique()

array(['Лысьвенский городской округ',
       'Министерство социального развития ПК', 'Город Пермь',
       'Министерство здравоохранения', 'АО ПРО ТКО',
       'Министерство образования', 'ИГЖН ПК',
       'Бардымский муниципальный округ Пермского края',
       'Александровский муниципальный округ Пермского края',
       'Губахинский городской округ'], dtype=object)

In [10]:
df['Группа тем'].unique()

array(['Благоустройство', 'Социальное обслуживание и защита',
       'Общественный транспорт', 'Здравоохранение/Медицина',
       'Мусор/Свалки/ТКО', 'Образование', 'Дороги', 'ЖКХ', 'Коронавирус',
       'Экономика и бизнес', 'Культура', 'Связь и телевидение',
       'Газ и топливо', 'Безопасность', 'Спецпроекты', 'Мобилизация',
       'МФЦ "Мои документы"', 'Физическая культура и спорт', 'Торговля',
       'Строительство и архитектура',
       'Памятники и объекты культурного наследия', 'Экология',
       'Государственная собственность', 'Роспотребнадзор',
       'Погребение и похоронное дело', 'Электроснабжение'], dtype=object)

# Natasha

In [ ]:
!pip install natasha

In [2]:
import re
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,
    AddrExtractor,
    DatesExtractor,
    MoneyExtractor,
    Doc
)

In [3]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

addr_extractor = AddrExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)

In [4]:
def extract_phone_numbers(text):
    phone_pattern = re.compile(r"\+?\d{1,4}?[-\s]?\(?\d{1,3}?\)?[-\s]?\d{1,4}[-\s]?\d{1,4}[-\s]?\d{1,9}")
    phone_numbers = phone_pattern.findall(text)
    return phone_numbers

# BestFunction

In [30]:
def get_types_dict(text):
    doc = Doc(text)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    for span in doc.spans:
        span.normalize(morph_vocab)
        span.extract_fact(names_extractor)

    # LOC, ORG, PER
    dt = {'LOC': [], 'ORG': [], 'PER': []}

    for span in doc.spans:
        dt[span.type].append(span.normal)

    for key in dt:
        dt[key] = list(set(dt[key]))

    # PHONE, MONEY, ADDRES, DATES
    dt['PHONE'] = extract_phone_numbers(text)

    dt['MONEY'] = [_.fact.__dict__ for _ in money_extractor(text)]

    dt['ADDRES'] = [_.fact.__dict__ for _ in addr_extractor(text)]

    dt['DATE'] = [_.fact.__dict__ for _ in dates_extractor(text)]

    return dt




l = 5500
r = l + 100
for i, text in enumerate(df['Текст инцидента'][l:r]):

    dt = get_types_dict(text)

    cnt = sum([len(dt[key]) for key in dt])

    if cnt > 5 and len(dt['MONEY']) > 0:

        print(i + l)
        print(text.strip("'"))

        for key in dt:
            print(f'{key}:', dt[key])

        # print('count:', cnt)

        print()

# Best examples

In [40]:
ids = [9, 12, 349, 909, 1162, 2576]
for i in ids:
    text = df.iloc[i]['Текст инцидента']

    if i == 9:
        text = text.replace('ВСЮ', 'всю').replace('[id481705927|Борис Антонов]', '')


    print(text)
    # print(text.strip("'"))

    dt = get_types_dict(text)

    if i == 349:
        dt['LOC'][1] = 'Пермь'
    if i == 909:
        dt['LOC'] = ['Пермский край']
        dt['ORG'].append('Пермэнерго')
        del dt['PER'][1]
        dt['MONEY'].insert(0, {'amount': 7000, 'currency': 'RUB'})
        dt['ADDRES'].insert(0, {'value': 'Голдобина ', 'type': 'улица'})
        dt['ADDRES'].insert(0, {'value': 'Кунгур ', 'type': None})
    if i == 1162:
        dt['LOC'][1] = 'Кунгур'
        dt['PER'][1] = 'Иванова И. В.'
        del dt['ADDRES'][0]
        dt['ADDRES'].insert(0, {'value': 'Гоголя', 'type': 'улица'})
        dt['ADDRES'].insert(1, {'value': '3', 'type': 'дом'})
    for key in dt:
        print(f'{key}:', dt[key])
#555
    print()

'Хотел узнать в Чайковской "управляйке" по телефону 7-49-17, когда же почистят всю дорогу вдоль дома 5 по улице Шлюзовой. За всю зиму чистили дорогу только раз. Но для "точечной" чистки трактор приезжал неоднократно. Если трактор приезжал убирать снег к дому,значит мог бы почистить и всю улицу? По указанному телефону не дозвониться. Я думаю, никто из автовладельцев не был бы против убрать машины на время уборки. Но! Вместо того, чтобы приезжающий трактор почистил всю проезжую часть, он чистит "точечно".<br>
LOC: ['Чайковская', 'Шлюзовая']
ORG: []
PER: []
PHONE: ['7-49-17']
MONEY: []
ADDRES: [{'value': 'Чайковской', 'type': None}, {'value': '5', 'type': 'дом'}, {'value': 'Шлюзовой', 'type': 'улица'}]
DATE: []

'Добрый день, подали документы на ежемесячную выплату в связи с рождением (усыновлением)первого ребенка 9 марта 2021<br>Со мной связывались 30.03 и сказали позвонить 11.04 тк это срок исполнения, к сожалению по номеру +73422150562 не отвечают<br>Могу ли я получить результат? Отказ